# Mini block Cipher Implementation
## Primitives

In [35]:
# Mini Block Cipher Implementation and Meet-in-the-Middle Attack

# --- Task 1: Implementing Mini Block Cipher ---

# Key Expansion: Split 16-bit key into Key0, Key1, Key2
def key_expansion(key):
    Key0 = (key >> 8) & 0xFF  # First 8 bits
    Key1 = key & 0xFF        # Next 8 bits
    Key2 = Key0 ^ Key1        # XOR of Key0 and Key1 (example)
    return Key0, Key1, Key2

# Substitution function (example: simple XOR with a constant)
# Create s-box (nib) table and the inverse
s_box = {
    0b0000: 0b1001,
    0b0001: 0b0100,
    0b0010: 0b1010,
    0b0011: 0b1011,
    0b0100: 0b1101,
    0b0101: 0b0001,
    0b0110: 0b1000,
    0b0111: 0b0101,
    0b1000: 0b0110,
    0b1001: 0b0010,
    0b1010: 0b0000,
    0b1011: 0b0011,
    0b1100: 0b1100,
    0b1101: 0b1110,
    0b1110: 0b1111,
    0b1111: 0b0111
}

inverse_s_box = {v: k for k, v in s_box.items()}

def substitute(state, inverse=False):
    n0 = (state >> 12) & 0x0F
    n1 = (state >> 8) & 0x0F
    n2 = (state >> 4) & 0x0F
    n3 = state & 0x0F
    if inverse:  # Reverse substitution (16-bit constant)
        s0 = inverse_s_box[n0]
        s1 = inverse_s_box[n1]
        s2 = inverse_s_box[n2]
        s3 = inverse_s_box[n3]
        return (s0 << 12) | (s1 << 8) | (s2 << 4) | s3
    else:  # Forward substitution (16-bit constant)
        s0 = s_box[n0]
        s1 = s_box[n1]
        s2 = s_box[n2]
        s3 = s_box[n3]
        return (s0 << 12) | (s1 << 8) | (s2 << 4) | s3

# Shift function (example: circular shift left by 4 bits)
# Since we are only swapping bottom row, there is no inverse shift
def shift(state, inverse=False):
    k0 = (state >> 12) & 0x0F
    k2 = (state >> 4) & 0x0F
    k1 = state & 0x0F
    k3 = (state >> 8) & 0x0F
    return (k0 << 12) | (k1 << 8) | (k2 << 4) | k3

# Mix function (example: XOR with a constant)
# XOR is it's own inverse
def mix(state, inverse=False):
    n0 = (state >> 12) & 0x0F
    n1 = (state >> 8) & 0x0F
    n2 = (state >> 4) & 0x0F
    n3 = state & 0x0F
    mixed_cols = 0x0
    for i, c in enumerate([n0 << 4 | n1, n2 << 4 | n3]):
        b0 = ((c & 0x10000000) >> 7) & 0x0F
        b1 = ((c & 0x01000000) >> 6) & 0x0F
        b2 = ((c & 0x00100000) >> 5) & 0x0F
        b3 = ((c & 0x00010000) >> 4) & 0x0F
        b4 = ((c & 0x00001000) >> 3) & 0x0F
        b5 = ((c & 0x00000100) >> 2) & 0x0F
        b6 = ((c & 0x00000010) >> 1) & 0x0F
        b7 = ((c & 0x00000001) >> 0) & 0x0F
        m0 = b0 ^ b6
        m1 = b1 ^ b4 ^ b7
        m2 = b2 ^ b4 ^ b5
        m3 = b3 ^ b5
        m4 = b2 ^ b4
        m5 = b0 ^ b3 ^ b5
        m6 = b0 ^ b1 ^ b6
        m7 = b1 ^ b7
        mixed_cols = mixed_cols << 8 | (m0 << i+7) | (m1 << i+6) | (m2 << i+5) | (m3 << i+4) | (m4 << i+3) | (m5 << i+2) | (m6 << i+1) | m7 << i+0
        return mixed_cols


### Test primitives
Using values from the slides

In [30]:
# Key Expansion
k = 0b0101100101111010
k0, k1, k2 = key_expansion(k)

print(f'k0: {k0:08b}, k1: {k1:08b}, k2: {k2:08b}')
assert k0 == 0b01011001
assert k1 == 0b01111010
assert k2 == k0 ^ k1

k0: 01011001, k1: 01111010, k2: 00100011


In [31]:
# Substitution
k = 0b0000001100111001
s = substitute(k)

print(f'state     : {k:016b}')
print(f'substitute: {s:016b}')
assert s == 0b1001101110110010

state     : 0000001100111001
substitute: 1001101110110010


In [38]:
# Shift row
k = 0b0000001100111001
s = substitute(k)
sr = shift(s)

print(f'k     : {k:016b}')
print(f's     : {s:016b}')
print(f'shift : {sr:016b}')
assert sr == 0b1001001010111011

k     : 0000001100111001
s     : 1001101110110010
shift : 1001001010111011


In [39]:
# Mix columns
k = 0b0000001100111001
s = substitute(k)
sr = shift(s)
m = mix(sr)
print(f'shift : {sr:016b}')
print(f'mix   : {m:016b}')
assert m == 0b0001000000010001

shift : 1001001010111011
mix   : 0000010000010000


AssertionError: 

## Encryption Functions

In [ ]:

# AddRoundKey function (XOR with the round key)
def add_round_key(state, round_key):
    return state ^ round_key

# Encryption Round 1
def encrypt_round1(state, Key1):
    state = substitute(state)
    state = shift(state)
    state = mix(state)
    state = add_round_key(state, Key1)
    return state

# Encryption Round 2
def encrypt_round2(state, Key2):
    state = substitute(state)
    state = shift(state)
    state = add_round_key(state, Key2)
    return state

# Decryption Round 2
def decrypt_round2(state, Key2):
    state = add_round_key(state, Key2)
    state = shift(state, inverse=True)
    state = substitute(state, inverse=True)
    return state

# Decryption Round 1
def decrypt_round1(state, Key1):
    state = add_round_key(state, Key1)
    state = mix(state, inverse=True)
    state = shift(state, inverse=True)
    state = substitute(state, inverse=True)
    return state

# Full Encryption
def encrypt(plaintext, key):
    Key0, Key1, Key2 = key_expansion(key)
    state = plaintext ^ Key0  # Initial AddRoundKey
    state = encrypt_round1(state, Key1)
    state = encrypt_round2(state, Key2)
    return state

# Full Decryption
def decrypt(ciphertext, key):
    Key0, Key1, Key2 = key_expansion(key)
    state = decrypt_round2(ciphertext, Key2)
    state = decrypt_round1(state, Key1)
    state = state ^ Key0  # Final AddRoundKey
    return state

# Implmentation

In [4]:

# Generate Plaintext-Ciphertext Pairs
key = 0b1010101010101010
plaintext_ciphertext_pairs = []
for i in range(10):
    plaintext = i * 0b1100110011001100
    ciphertext = encrypt(plaintext, key)
    plaintext_ciphertext_pairs.append((plaintext, ciphertext))

print("Plaintext-Ciphertext Pairs:")
for p, c in plaintext_ciphertext_pairs:
    print(f"Plaintext: {p:016b}, Ciphertext: {c:016b}")

# --- Task 2: Meet-in-the-Middle Attack ---

def meet_in_the_middle(plaintext_ciphertext_pairs):
    (plaintext1, ciphertext1), (plaintext2, ciphertext2) = plaintext_ciphertext_pairs[:2] #use the first two pairs.

    forward_table = {}
    for key1 in range(256):  # Key1 is 8 bits
        intermediate = encrypt_round1(plaintext1, key1)
        forward_table[intermediate] = key1

    reverse_table = {}
    for key2 in range(256):  # Key2 is 8 bits
        intermediate = decrypt_round2(ciphertext1, key2)
        if intermediate in forward_table:
            reverse_table.setdefault(intermediate, []).append(key2)

    possible_keys = []
    for intermediate, key1 in forward_table.items():
        if intermediate in reverse_table:
            for key2 in reverse_table[intermediate]:
                # Verify with the second pair
                if encrypt_round2(encrypt_round1(plaintext2, key1), key2) == ciphertext2:
                    possible_keys.append((key1, key2))

    return possible_keys

possible_keys = meet_in_the_middle(plaintext_ciphertext_pairs)
print("\nPossible Key Pairs:")
for k1, k2 in possible_keys:
    print(f"Key1: {k1:08b}, Key2: {k2:08b}")

# --- Task 3: Analysis ---

# a) Key Space
key_space = 2**16
print(f"\nKey Space: {key_space}")

# b) Double Mini Block Cipher MITM Attack
mitm_operations_double = 2 * (2**16) # 2^16 encryption + 2^16 decryption
print(f"MITM Operations (Double Mini): {mitm_operations_double}")

# c) Exhaustive Key Search (Double Mini)
exhaustive_operations_double = 2**32
print(f"Exhaustive Operations (Double Mini): {exhaustive_operations_double}")

# d) Tradeoff
print("\nMITM Attack Tradeoff:")
print("Speed: MITM is significantly faster than exhaustive search for multiple rounds.")
print("Memory: MITM requires storing intermediate results, increasing memory usage.")
print("Complexity: MITM reduces time complexity from O(2^2n) to O(2^n) for two rounds.")

state: 0000000010101010


KeyError: 170

In [96]:
# Mini Block Cipher Implementation and Meet-in-the-Middle Attack

# --- Task 1: Implementing Mini Block Cipher ---

# Key Expansion: Split 16-bit key into Key0, Key1, Key2
def key_expansion(key):
    Key0 = (key >> 8) & 0xFF  # First 8 bits
    Key1 = key & 0xFF        # Next 8 bits
    Key2 = Key0 ^ Key1        # XOR of Key0 and Key1 (example)
    return Key0, Key1, Key2

# Substitution function (example: simple XOR with a constant)
def substitute(state, inverse=False):
    if inverse:
        return state ^ 0b1010101010101010  # Inverse substitution (16-bit constant)
    return state ^ 0b0101010101010101      # Forward substitution (16-bit constant)

# Shift function (example: circular shift left by 4 bits)
def shift(state, inverse=False):
    if inverse:
        return ((state >> 4) | (state << 12)) & 0xFFFF  # Reverse shift (16-bit)
    return ((state << 4) | (state >> 12)) & 0xFFFF      # Forward shift (16-bit)

# Mix function (example: XOR with a constant)
def mix(state, inverse=False):
    if inverse:
        return state ^ 0b1100110011001100  # Inverse mix (16-bit constant)
    return state ^ 0b0011001100110011      # Forward mix (16-bit constant)

# AddRoundKey function (XOR with the round key)
def add_round_key(state, round_key):
    return state ^ round_key

# Encryption Round 1
def encrypt_round1(state, Key1):
    print(f"  Round 1 - Input State: {state:016b}")
    state = substitute(state)
    print(f"  Round 1 - After Substitute: {state:016b}")
    state = shift(state)
    print(f"  Round 1 - After Shift: {state:016b}")
    state = mix(state)
    print(f"  Round 1 - After Mix: {state:016b}")
    state = add_round_key(state, Key1)
    print(f"  Round 1 - After AddRoundKey: {state:016b}")
    return state

# Encryption Round 2
def encrypt_round2(state, Key2):
    print(f"  Round 2 - Input State: {state:016b}")
    state = substitute(state)
    print(f"  Round 2 - After Substitute: {state:016b}")
    state = shift(state)
    print(f"  Round 2 - After Shift: {state:016b}")
    state = add_round_key(state, Key2)
    print(f"  Round 2 - After AddRoundKey: {state:016b}")
    return state

# Decryption Round 2
def decrypt_round2(state, Key2):
    print(f"  Decryption Round 2 - Input State: {state:016b}")
    state = add_round_key(state, Key2)
    print(f"  Decryption Round 2 - After AddRoundKey: {state:016b}")
    state = shift(state, inverse=True)
    print(f"  Decryption Round 2 - After Shift: {state:016b}")
    state = substitute(state, inverse=True)
    print(f"  Decryption Round 2 - After Substitute: {state:016b}")
    return state

# Decryption Round 1
def decrypt_round1(state, Key1):
    print(f"  Decryption Round 1 - Input State: {state:016b}")
    state = add_round_key(state, Key1)
    print(f"  Decryption Round 1 - After AddRoundKey: {state:016b}")
    state = mix(state, inverse=True)
    print(f"  Decryption Round 1 - After Mix: {state:016b}")
    state = shift(state, inverse=True)
    print(f"  Decryption Round 1 - After Shift: {state:016b}")
    state = substitute(state, inverse=True)
    print(f"  Decryption Round 1 - After Substitute: {state:016b}")
    return state

# Full Encryption
def encrypt(plaintext, key):
    Key0, Key1, Key2 = key_expansion(key)
    print(f"Key0: {Key0:08b}, Key1: {Key1:08b}, Key2: {Key2:08b}")
    state = plaintext ^ Key0  # Initial AddRoundKey
    print(f"Initial AddRoundKey: {state:016b}")
    state = encrypt_round1(state, Key1)
    state = encrypt_round2(state, Key2)
    print(f"Final Ciphertext: {state:016b}")
    return state

# Full Decryption
def decrypt(ciphertext, key):
    Key0, Key1, Key2 = key_expansion(key)
    print(f"Key0: {Key0:08b}, Key1: {Key1:08b}, Key2: {Key2:08b}")
    state = decrypt_round2(ciphertext, Key2)
    state = decrypt_round1(state, Key1)
    state = state ^ Key0  # Final AddRoundKey
    print(f"Final Plaintext: {state:016b}")
    return state

# Generate Plaintext-Ciphertext Pairs
key = 0b1010101010101010
plaintext_ciphertext_pairs = []
for i in range(10):
    plaintext = i * 0b1100110011001100
    ciphertext = encrypt(plaintext, key)
    plaintext_ciphertext_pairs.append((plaintext, ciphertext))

print("Plaintext-Ciphertext Pairs:")
for p, c in plaintext_ciphertext_pairs:
    print(f"Plaintext: {p:016b}, Ciphertext: {c:016b}")

# --- Task 2: Meet-in-the-Middle Attack ---

def meet_in_the_middle(plaintext_ciphertext_pairs):
    (plaintext1, ciphertext1), (plaintext2, ciphertext2) = plaintext_ciphertext_pairs[:2] #use the first two pairs.

    forward_table = {}
    for key1 in range(256):  # Key1 is 8 bits
        intermediate = encrypt_round1(plaintext1, key1)
        forward_table[intermediate] = key1

    reverse_table = {}
    for key2 in range(256):  # Key2 is 8 bits
        intermediate = decrypt_round2(ciphertext1, key2)
        if intermediate in forward_table:
            reverse_table.setdefault(intermediate, []).append(key2)

    possible_keys = []
    for intermediate, key1 in forward_table.items():
        if intermediate in reverse_table:
            for key2 in reverse_table[intermediate]:
                # Verify with the second pair
                if encrypt_round2(encrypt_round1(plaintext2, key1), key2) == ciphertext2:
                    possible_keys.append((key1, key2))

    return possible_keys

possible_keys = meet_in_the_middle(plaintext_ciphertext_pairs)
print("\nPossible Key Pairs:")
for k1, k2 in possible_keys:
    print(f"Key1: {k1:08b}, Key2: {k2:08b}")

# --- Task 3: Analysis ---

# a) Key Space
key_space = 2**16
print(f"\nKey Space: {key_space}")

# b) Double Mini Block Cipher MITM Attack
mitm_operations_double = 2 * (2**16) # 2^16 encryption + 2^16 decryption
print(f"MITM Operations (Double Mini): {mitm_operations_double}")

# c) Exhaustive Key Search (Double Mini)
exhaustive_operations_double = 2**32
print(f"Exhaustive Operations (Double Mini): {exhaustive_operations_double}")

Key0: 10101010, Key1: 10101010, Key2: 00000000
Initial AddRoundKey: 0000000010101010
  Round 1 - Input State: 0000000010101010
  Round 1 - After Substitute: 0101010111111111
  Round 1 - After Shift: 0101111111110101
  Round 1 - After Mix: 0110110011000110
  Round 1 - After AddRoundKey: 0110110001101100
  Round 2 - Input State: 0110110001101100
  Round 2 - After Substitute: 0011100100111001
  Round 2 - After Shift: 1001001110010011
  Round 2 - After AddRoundKey: 1001001110010011
Final Ciphertext: 1001001110010011
Key0: 10101010, Key1: 10101010, Key2: 00000000
Initial AddRoundKey: 1100110001100110
  Round 1 - Input State: 1100110001100110
  Round 1 - After Substitute: 1001100100110011
  Round 1 - After Shift: 1001001100111001
  Round 1 - After Mix: 1010000000001010
  Round 1 - After AddRoundKey: 1010000010100000
  Round 2 - Input State: 1010000010100000
  Round 2 - After Substitute: 1111010111110101
  Round 2 - After Shift: 0101111101011111
  Round 2 - After AddRoundKey: 0101111101011111